In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from common import *
from PatientVec.Experiments.hyperparam_exps import *

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [ ]:
from dataloaders import mortality_dataset
data = mortality_dataset(args, _type='30day')

In [ ]:
train_data, dev_data = get_basic_data(data, structured=True, truncate=90)

In [ ]:
config = {'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 
          'type' : data.metrics_type, 'norm' : 'l2', 'constant_mul' : 1.0}
lr = LR(config)
lr.train(train_data)
metrics = lr.evaluate(dev_data, save_results=True)

Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['roc_auc', 'pr_auc']
generate_latex_tables(data, keys_to_use)

In [ ]:
dirname = 'outputs/Readmission/Basic/'
exps = os.listdir(dirname)
for e in sorted(exps) :
    if 'Structured' in e :
        print(e)
        print_results_from_model(get_latest_model(os.path.join(dirname, e)))

In [ ]:
dirname = 'outputs/Diagnosis/Basic/'
exps = os.listdir(dirname)
for e in sorted(exps) :
    if 'Structured' in e :
        print(e)
        print_results_from_model(get_latest_model(os.path.join(dirname, e)))

In [ ]:
keys = ['roc_auc', 'pr_auc']
yr_df = defaultdict(list)
for yr in [1] :
    for model in ['baselines', 'Basic', 'Attention', 'Structured Attention'] :
        name = os.path.join('outputs/Mortality_30day/', model)
        dirs = os.listdir(name)
        dfs = {}
        for d in sorted(dirs) :
            dirname = os.path.join(name, d)
            model_name = get_latest_model(dirname)
            if model_name is not None :
                df = json.load(open(os.path.join(model_name, 'evaluate.json')))
                dfs[d] = {k:v for k, v in df.items() if k in keys}
        
        dfs = pd.DataFrame(dfs).transpose()
        yr_df[yr] += [dfs]
        
    yr_df[yr] = pd.concat(yr_df[yr], axis=0)
yr_df = pd.concat(yr_df.values(), axis=1, keys=yr_df.keys()).round(3)
yr_df.columns = yr_df.columns.swaplevel(0, 1)
yr_df.sort_index(axis=1, level=0, inplace=True)
display(HTML(yr_df.to_html()))